In [25]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

In [26]:
#Path of the file with the data of particles
path = r'/home/user/Documents/BUAP/Estancia/CosmicRayReco/MCAnalysis/ClusterAnalysis/Data'
os.chdir(path)

In [27]:
#File with summary of Showers
Showers = pd.read_csv('DAT000006_showers.txt',delimiter='\t')

In [28]:
#File with information of all particles at observation level
Particles = pd.read_csv('DAT000006_particles.txt',delimiter='\t')

In [29]:
#Change the units of first interaction point from cm to km
Showers['Zfirst'] = Showers['Zfirst'].mul(1e-5)

KeyError: 'Zfirst'

In [6]:
#change units of positions from cm to m
Particles[['x','y']] = Particles[['x','y']].mul(0.01)

In [7]:
#Statistics of the data frame with summary of showers 
Showers.describe()

,Shower,Energy,Zfirst,Theta,Phi,Particles
count,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,1000.0,21.396340,0.292903,0.032679,76.248000
std,288.819436,0.0,8.507688,0.106816,1.807978,95.304866
min,1.000000,1000.0,5.456020,0.017145,-3.132620,6.000000
25%,250.750000,1000.0,15.546450,0.215487,-1.581553,29.000000
50%,500.500000,1000.0,19.986050,0.308449,0.029519,43.000000
75%,750.250000,1000.0,25.346650,0.384153,1.675545,79.000000
max,1000.000000,1000.0,69.751400,0.450248,3.137550,896.000000


In [8]:
#Definition of a box which contain all the particles of the simulation
#size of the detector on mm (¡¡¡For the moment work with a detector of 1m^2 !!!)

det_s_x = 1 # detector size in x axis (m)
det_s_y = 1 # detector size in y axis (m)


In [9]:
Particles['DetPosx'] = 0; Particles['DetPosy'] = 0

In [10]:
#The values in the 'DetPosx' and 'DetPosy' columns will be replaced by the rounded values 
#of the positions of the particles, this will define the position of the cell
#For the X axis
Particles.loc[Particles['x'] > 0,'DetPosx'] = np.ceil(Particles.loc[Particles['x'] > 0,'x'])
Particles.loc[Particles['x'] < 0,'DetPosx'] = np.floor(Particles.loc[Particles['x'] < 0,'x'])
#For the Y axis
Particles.loc[Particles['y'] > 0,'DetPosy'] = np.ceil(Particles.loc[Particles['y'] > 0,'y'])
Particles.loc[Particles['y'] < 0,'DetPosy'] = np.floor(Particles.loc[Particles['y'] < 0,'y'])

In [11]:
#The values in the 'CoorDetx' and 'CoorDety' columns will be replaced by the division remainder values
#of the positions of the particles divided by the size of the detector, this will define the position 
#in the cell
#For the X axis
Particles['CoorDetx'] = Particles['x'] % det_s_x
#For the Y axis
Particles['CoorDety'] = Particles['y'] % det_s_y
#Add a new column which will contain the number of cluster 
Particles['Ncluster'] = 0

In [12]:
Particles

,sh,id,x,y,t,px,py,pz,Psq,ek,zha,aza,DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster
0,1,1,202.8570,-52.80860,70347.8,-0.014051,0.000221,0.052103,0.002912,0.053965,0.263442,3.12588,203,-53,0.8570,0.19140,0
1,1,1,43.9967,-18.30410,70512.6,-0.182562,0.015446,0.487750,0.271468,0.521026,0.359324,3.05719,44,-19,0.9967,0.69590,0
2,1,1,26.9723,-5.68421,70534.3,-0.126190,0.013492,0.323988,0.121074,0.347957,0.373339,3.03508,27,-6,0.9723,0.31579,0
3,1,1,33.2305,3.98663,70528.2,-0.020200,0.002715,0.052752,0.003198,0.056553,0.368697,3.00797,34,4,0.2305,0.98663,0
4,1,1,79.4011,13.43910,70477.4,-0.018291,0.003078,0.052332,0.003083,0.055522,0.340619,2.97490,80,14,0.4011,0.43910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76243,1000,5,60.9239,15.21820,66522.0,-33.247300,-11.449000,106.593000,12598.600000,112.138000,0.318642,-2.80995,61,16,0.9239,0.21820,0
76244,1000,2,-356.6540,360.43600,66859.6,-0.014904,-0.009428,0.052701,0.003088,0.055065,0.322918,-2.57752,-357,361,0.3460,0.43600,0
76245,1000,5,484.7020,354.32500,66030.5,-1.106890,-0.376404,3.540170,13.899700,3.624060,0.318973,-2.81381,485,355,0.7020,0.32500,0
76246,1000,5,60.6589,-90.33680,66559.4,-9.388220,-3.536470,30.083000,1005.630000,31.606300,0.321886,-2.78134,61,-91,0.6589,0.66320,0


In [13]:
#Define a function to find the clusters of particles
def Clusters(ith_clust, Sh_n):
    
    # Transform the columns of the particle data frame with the position of the detector into a list
    x_pos = Sh_n['DetPosx'].tolist()
    y_pos = Sh_n['DetPosy'].tolist()
    
    # Create an empty set
    unique_pairs = set()

    # Iterate over the pairs of x and y using zip
    for xi, yi in zip(x_pos, y_pos):
        pair = (xi, yi)

        # Check if the pair is already in the set
        if pair not in unique_pairs:
            # If not, add it to the set
            unique_pairs.add(pair)

            # Extract the particles index by comparing it with the x, y position in the data frame
            p_index_clust = Sh_n.index[ (Sh_n['DetPosx'] == xi) & (Sh_n['DetPosy'] == yi) ].tolist()
            
            # Replacte the cluster position of the particle for the cluster id
            for j in range(len(p_index_clust)):
                Sh_n.at[p_index_clust[j],'Ncluster'] = ith_clust

            ith_clust = ith_clust+1 # Next Cluster
    
    return Sh_n, ith_clust 
            

In [14]:
%%time
ith_clust = 1 # First cluster

# Run over all the showers
for ith_shower in range(1, Showers['Shower'].count()+1):
    Sh_n, ith_clust = Clusters(ith_clust, Particles[Particles['sh']==ith_shower]) 
    Particles.loc[Particles['sh']==ith_shower, :] = Sh_n

CPU times: user 1min 22s, sys: 168 ms, total: 1min 22s
Wall time: 1min 22s


In [15]:
# Check if there are particles that are not counted
Particles[Particles["Ncluster"]==0]

,sh,id,x,y,t,px,py,pz,Psq,ek,zha,aza,DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster


In [16]:
# Number of the cluster with the maximum number of particles
df_sn = Particles[Particles["Ncluster"]!=0]
print(df_sn["Ncluster"].value_counts().idxmax())

24354


In [17]:
# Summary of the cluster with the maximum number of particles
Particles[Particles["Ncluster"]==24354].describe()

,sh,id,x,y,t,px,py,pz,Psq,ek,zha,aza,DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster
count,27.0,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.0,27.0,27.000000,27.000000,27.0
mean,328.0,1.370370,1.757474,15.579659,18024.892593,0.285113,0.205996,1.121763,5.039791,1.175498,0.304319,0.620638,2.0,16.0,0.757474,0.579659,24354.0
std,0.0,0.687702,0.204971,0.225280,0.156711,0.471210,0.342276,1.859893,18.289401,1.948820,0.007892,0.018561,0.0,0.0,0.204971,0.225280,0.0
min,328.0,1.000000,1.279970,15.180800,18024.600000,0.013336,0.009632,0.052452,0.003022,0.054971,0.279058,0.559420,2.0,16.0,0.279970,0.180800,24354.0
25%,328.0,1.000000,1.607505,15.457000,18024.850000,0.034435,0.023948,0.139446,0.021206,0.145365,0.303065,0.623762,2.0,16.0,0.607505,0.457000,24354.0
50%,328.0,1.000000,1.811220,15.543800,18024.900000,0.156331,0.112325,0.615420,0.415798,0.644824,0.303589,0.626550,2.0,16.0,0.811220,0.543800,24354.0
75%,328.0,1.500000,1.919385,15.750950,18025.000000,0.278946,0.202618,1.098910,1.327010,1.151725,0.304003,0.627727,2.0,16.0,0.919385,0.750950,24354.0
max,328.0,3.000000,1.999420,15.935900,18025.300000,2.357560,1.712170,9.305530,95.082600,9.750520,0.327312,0.639445,2.0,16.0,0.999420,0.935900,24354.0


In [18]:
Particles[Particles["Ncluster"]==24354]

,sh,id,x,y,t,px,py,pz,Psq,ek,zha,aza,DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster
25576,328,1,1.97012,15.9225,18025.3,0.017121,0.012351,0.067537,0.005007,0.070760,0.302958,0.624956,2,16,0.97012,0.9225,24354
25595,328,1,1.27997,15.5098,18024.6,0.033219,0.021118,0.115938,0.014991,0.122438,0.327312,0.566283,2,16,0.27997,0.5098,24354
25596,328,2,1.27997,15.5085,18024.6,0.136209,0.085286,0.477266,0.253610,0.503086,0.324798,0.559420,2,16,0.27997,0.5085,24354
25659,328,1,1.87157,15.5438,18025.0,0.017631,0.012707,0.069351,0.005282,0.072677,0.303684,0.624484,2,16,0.87157,0.5438,24354
25660,328,1,1.74490,15.5494,18024.9,0.444662,0.321847,1.755030,3.381460,1.838870,0.303128,0.626521,2,16,0.74490,0.5494,24354
25661,328,1,1.75018,15.5706,18024.9,0.239785,0.173728,0.946251,0.983070,0.991499,0.303272,0.626992,2,16,0.75018,0.5706,24354
25662,328,2,1.91793,15.5637,18025.0,0.790821,0.567981,3.074410,10.400000,3.224390,0.306703,0.622847,2,16,0.91793,0.5637,24354
25663,328,1,1.81122,15.7059,18025.0,0.013336,0.009632,0.052452,0.003022,0.054971,0.303922,0.625491,2,16,0.81122,0.7059,24354
25664,328,1,1.83963,15.7272,18025.0,0.095424,0.068919,0.375200,0.154631,0.393231,0.304000,0.625495,2,16,0.83963,0.7272,24354
25665,328,1,1.59644,15.7747,18024.9,0.092728,0.067197,0.365548,0.146739,0.383066,0.303589,0.627088,2,16,0.59644,0.7747,24354


In [21]:
Particles[Particles["Ncluster"]==200]

,sh,id,x,y,t,px,py,pz,Psq,ek,zha,aza,DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster
209,1,6,127.962,-127.648,70421.4,-0.932691,0.049322,2.57142,7.48456,2.63217,0.348404,3.08876,128,-128,0.962,0.352,200
